In [1]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/INFOSYS765/yelp_review.csv')

In [4]:
df.head()

,stars,useful,text,funny,review_id,cool,date,user_id,business_id
0,4.0,0,First time waxing. I read all the other review...,0,V46usAbOSPIzurZy7DUU5A,0,2016-04-16 13:55:38,rD6noiuiMCcGuq6oPYFJdQ,b4NaB1u3AwgrhRMKsKgWsA
1,5.0,0,I love my manicure! Such an affordable place. ...,0,CpNSsju9NFc-Xrpf5Hfqhg,0,2020-01-18 05:32:54,tTsUSklzH1oxPp2usth84A,-otKp6RA94dfGsqa5jSDHQ
2,4.0,0,"Great food, fast service, they try to crank pe...",0,RP7A24xD14Cbegmr1AokNQ,0,2015-01-25 14:49:09,-cnIq2OmV8CleS4bLOvCxQ,EA10YCkaxzRoCEOB38xv1Q
3,1.0,0,Save your money and go across the street the f...,0,6a38lS1TjeWMqK-knuNAhA,0,2018-11-06 22:12:14,mzCkRflEka_vezCiBYYWGA,nkqaLepNTSrUYuZkzbBR5A
4,4.0,0,Oops...this was for the Main St location:\n\n-...,0,Q0QKTuceHzgmFeTtYSTzYg,0,2011-08-19 21:45:31,3zlK9eBVCpRWMPLjtAj6cQ,XtFhLvKV46H7a054X2_aFQ


# Step 1: Change all text into lower-case
This step converts all text into lowercase

In [5]:
# a list to store letters in lowercase
lower_case = []

# converts each word in the text to lowercase
for word in df['text']:
  lower_case.append(''.join(word.lower()))
df['lower_case'] = pd.Series(lower_case)

In [6]:
df.head()

,stars,useful,text,funny,review_id,cool,date,user_id,business_id,lower_case
0,4.0,0,First time waxing. I read all the other review...,0,V46usAbOSPIzurZy7DUU5A,0,2016-04-16 13:55:38,rD6noiuiMCcGuq6oPYFJdQ,b4NaB1u3AwgrhRMKsKgWsA,first time waxing. i read all the other review...
1,5.0,0,I love my manicure! Such an affordable place. ...,0,CpNSsju9NFc-Xrpf5Hfqhg,0,2020-01-18 05:32:54,tTsUSklzH1oxPp2usth84A,-otKp6RA94dfGsqa5jSDHQ,i love my manicure! such an affordable place. ...
2,4.0,0,"Great food, fast service, they try to crank pe...",0,RP7A24xD14Cbegmr1AokNQ,0,2015-01-25 14:49:09,-cnIq2OmV8CleS4bLOvCxQ,EA10YCkaxzRoCEOB38xv1Q,"great food, fast service, they try to crank pe..."
3,1.0,0,Save your money and go across the street the f...,0,6a38lS1TjeWMqK-knuNAhA,0,2018-11-06 22:12:14,mzCkRflEka_vezCiBYYWGA,nkqaLepNTSrUYuZkzbBR5A,save your money and go across the street the f...
4,4.0,0,Oops...this was for the Main St location:\n\n-...,0,Q0QKTuceHzgmFeTtYSTzYg,0,2011-08-19 21:45:31,3zlK9eBVCpRWMPLjtAj6cQ,XtFhLvKV46H7a054X2_aFQ,oops...this was for the main st location:\n\n-...


# Step 2: Remove digits
The we throw the digits from the text since they are unnecessary in a NLP model. We drop these digits by checking if a letter is a digit or not.

In [7]:
# a list to store non-digit words
non_digits = []

# loops through each sentence in the 'lower_case' column and then
# keeps it in the letters list if it is non-digit
for word in df['lower_case']:
  letters = [letter for letter in word if not letter.isdigit()]
  non_digits.append(''.join(letters))

df['text_non_digits'] = pd.Series(non_digits)
df.head()

,stars,useful,text,funny,review_id,cool,date,user_id,business_id,lower_case,text_non_digits
0,4.0,0,First time waxing. I read all the other review...,0,V46usAbOSPIzurZy7DUU5A,0,2016-04-16 13:55:38,rD6noiuiMCcGuq6oPYFJdQ,b4NaB1u3AwgrhRMKsKgWsA,first time waxing. i read all the other review...,first time waxing. i read all the other review...
1,5.0,0,I love my manicure! Such an affordable place. ...,0,CpNSsju9NFc-Xrpf5Hfqhg,0,2020-01-18 05:32:54,tTsUSklzH1oxPp2usth84A,-otKp6RA94dfGsqa5jSDHQ,i love my manicure! such an affordable place. ...,i love my manicure! such an affordable place. ...
2,4.0,0,"Great food, fast service, they try to crank pe...",0,RP7A24xD14Cbegmr1AokNQ,0,2015-01-25 14:49:09,-cnIq2OmV8CleS4bLOvCxQ,EA10YCkaxzRoCEOB38xv1Q,"great food, fast service, they try to crank pe...","great food, fast service, they try to crank pe..."
3,1.0,0,Save your money and go across the street the f...,0,6a38lS1TjeWMqK-knuNAhA,0,2018-11-06 22:12:14,mzCkRflEka_vezCiBYYWGA,nkqaLepNTSrUYuZkzbBR5A,save your money and go across the street the f...,save your money and go across the street the f...
4,4.0,0,Oops...this was for the Main St location:\n\n-...,0,Q0QKTuceHzgmFeTtYSTzYg,0,2011-08-19 21:45:31,3zlK9eBVCpRWMPLjtAj6cQ,XtFhLvKV46H7a054X2_aFQ,oops...this was for the main st location:\n\n-...,oops...this was for the main st location:\n\n-...


In [8]:
# validates that no digits exists in the text
a = 0
for word in df['text_non_digits']:
  for letters in word:
    if letters.isdigit() == True:
      a = a+1

print(a)

0


# Step 3: Remove the punctuation
The `text_non_digits` text has several punctuations. Punctuations are often unnecessary as it doesn’t add value or meaning to the NLP model. The “string” library has 32 punctuations.

Now, to remove punctuation, we would need to iterate through each word in the `text_non_digits` text, then check if the word is punctutation or not. If it is, then throw, otherwise keep it. At the end, join all of the words which are not punctuations. 


In [9]:
# a list to store non-punctuation words
non_punct = []

# loops through each sentence in the `text_non_digits` column and then checks each word in the text
# keeps it in the letters list if non-punctuation
for word in df['text_non_digits']:
  letters = [letter for letter in word if letter not in string.punctuation]
  non_punct.append(''.join(letters))

df['text_non_punct'] = pd.Series(non_punct)
df.head()

,stars,useful,text,funny,review_id,cool,date,user_id,business_id,lower_case,text_non_digits,text_non_punct
0,4.0,0,First time waxing. I read all the other review...,0,V46usAbOSPIzurZy7DUU5A,0,2016-04-16 13:55:38,rD6noiuiMCcGuq6oPYFJdQ,b4NaB1u3AwgrhRMKsKgWsA,first time waxing. i read all the other review...,first time waxing. i read all the other review...,first time waxing i read all the other reviews...
1,5.0,0,I love my manicure! Such an affordable place. ...,0,CpNSsju9NFc-Xrpf5Hfqhg,0,2020-01-18 05:32:54,tTsUSklzH1oxPp2usth84A,-otKp6RA94dfGsqa5jSDHQ,i love my manicure! such an affordable place. ...,i love my manicure! such an affordable place. ...,i love my manicure such an affordable place th...
2,4.0,0,"Great food, fast service, they try to crank pe...",0,RP7A24xD14Cbegmr1AokNQ,0,2015-01-25 14:49:09,-cnIq2OmV8CleS4bLOvCxQ,EA10YCkaxzRoCEOB38xv1Q,"great food, fast service, they try to crank pe...","great food, fast service, they try to crank pe...",great food fast service they try to crank peop...
3,1.0,0,Save your money and go across the street the f...,0,6a38lS1TjeWMqK-knuNAhA,0,2018-11-06 22:12:14,mzCkRflEka_vezCiBYYWGA,nkqaLepNTSrUYuZkzbBR5A,save your money and go across the street the f...,save your money and go across the street the f...,save your money and go across the street the f...
4,4.0,0,Oops...this was for the Main St location:\n\n-...,0,Q0QKTuceHzgmFeTtYSTzYg,0,2011-08-19 21:45:31,3zlK9eBVCpRWMPLjtAj6cQ,XtFhLvKV46H7a054X2_aFQ,oops...this was for the main st location:\n\n-...,oops...this was for the main st location:\n\n-...,oopsthis was for the main st location\n\n\n\nw...


# Step 4: Tokenization

Tokenizing is the process of splitting strings into a list of words. We will make use of Regular Expressions or regex to do the splitting. Regex can be used to describe a search pattern.

In [10]:
# a list to store tokens
tokens = []

# loops through each sentence in the `text_non_punct` column and then 
# splits on one or more non-word character and then 
for word in df['text_non_punct']: 
  split_words = re.split("\W+", word)
  tokens.append([word for word in split_words])

df['text_non_punct_split'] = pd.Series(tokens)
df.head()

,stars,useful,text,funny,review_id,cool,date,user_id,business_id,lower_case,text_non_digits,text_non_punct,text_non_punct_split
0,4.0,0,First time waxing. I read all the other review...,0,V46usAbOSPIzurZy7DUU5A,0,2016-04-16 13:55:38,rD6noiuiMCcGuq6oPYFJdQ,b4NaB1u3AwgrhRMKsKgWsA,first time waxing. i read all the other review...,first time waxing. i read all the other review...,first time waxing i read all the other reviews...,"[first, time, waxing, i, read, all, the, other..."
1,5.0,0,I love my manicure! Such an affordable place. ...,0,CpNSsju9NFc-Xrpf5Hfqhg,0,2020-01-18 05:32:54,tTsUSklzH1oxPp2usth84A,-otKp6RA94dfGsqa5jSDHQ,i love my manicure! such an affordable place. ...,i love my manicure! such an affordable place. ...,i love my manicure such an affordable place th...,"[i, love, my, manicure, such, an, affordable, ..."
2,4.0,0,"Great food, fast service, they try to crank pe...",0,RP7A24xD14Cbegmr1AokNQ,0,2015-01-25 14:49:09,-cnIq2OmV8CleS4bLOvCxQ,EA10YCkaxzRoCEOB38xv1Q,"great food, fast service, they try to crank pe...","great food, fast service, they try to crank pe...",great food fast service they try to crank peop...,"[great, food, fast, service, they, try, to, cr..."
3,1.0,0,Save your money and go across the street the f...,0,6a38lS1TjeWMqK-knuNAhA,0,2018-11-06 22:12:14,mzCkRflEka_vezCiBYYWGA,nkqaLepNTSrUYuZkzbBR5A,save your money and go across the street the f...,save your money and go across the street the f...,save your money and go across the street the f...,"[save, your, money, and, go, across, the, stre..."
4,4.0,0,Oops...this was for the Main St location:\n\n-...,0,Q0QKTuceHzgmFeTtYSTzYg,0,2011-08-19 21:45:31,3zlK9eBVCpRWMPLjtAj6cQ,XtFhLvKV46H7a054X2_aFQ,oops...this was for the main st location:\n\n-...,oops...this was for the main st location:\n\n-...,oopsthis was for the main st location\n\n\n\nw...,"[oopsthis, was, for, the, main, st, location, ..."


The `text_non_punct_split` column after step 
2 has created a list, by splitting all the non-word characters.

# Step 5: Remove stop words

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
stopword = nltk.corpus.stopwords.words('english')
# pring the first 11 stop words in the list
print(stopword[:11])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've"]


In [13]:
# a list to store stopwords
stopwords = []

# loops through each sentence in the `text_non_punct_split` column and then 
# stores words in a list of non_stop if the word is not a stopword
# then finally appends each list into stopwords
for text in df['text_non_punct_split']:
  non_stop = [word for word in text if word not in stopword]
  stopwords.append(non_stop)

df['text_non_punct_split_non_stopwords'] = pd.Series(stopwords)
df.head()

,stars,useful,text,funny,review_id,cool,date,user_id,business_id,lower_case,text_non_digits,text_non_punct,text_non_punct_split,text_non_punct_split_non_stopwords
0,4.0,0,First time waxing. I read all the other review...,0,V46usAbOSPIzurZy7DUU5A,0,2016-04-16 13:55:38,rD6noiuiMCcGuq6oPYFJdQ,b4NaB1u3AwgrhRMKsKgWsA,first time waxing. i read all the other review...,first time waxing. i read all the other review...,first time waxing i read all the other reviews...,"[first, time, waxing, i, read, all, the, other...","[first, time, waxing, read, reviews, decided, ..."
1,5.0,0,I love my manicure! Such an affordable place. ...,0,CpNSsju9NFc-Xrpf5Hfqhg,0,2020-01-18 05:32:54,tTsUSklzH1oxPp2usth84A,-otKp6RA94dfGsqa5jSDHQ,i love my manicure! such an affordable place. ...,i love my manicure! such an affordable place. ...,i love my manicure such an affordable place th...,"[i, love, my, manicure, such, an, affordable, ...","[love, manicure, affordable, place, manicurist..."
2,4.0,0,"Great food, fast service, they try to crank pe...",0,RP7A24xD14Cbegmr1AokNQ,0,2015-01-25 14:49:09,-cnIq2OmV8CleS4bLOvCxQ,EA10YCkaxzRoCEOB38xv1Q,"great food, fast service, they try to crank pe...","great food, fast service, they try to crank pe...",great food fast service they try to crank peop...,"[great, food, fast, service, they, try, to, cr...","[great, food, fast, service, try, crank, peopl..."
3,1.0,0,Save your money and go across the street the f...,0,6a38lS1TjeWMqK-knuNAhA,0,2018-11-06 22:12:14,mzCkRflEka_vezCiBYYWGA,nkqaLepNTSrUYuZkzbBR5A,save your money and go across the street the f...,save your money and go across the street the f...,save your money and go across the street the f...,"[save, your, money, and, go, across, the, stre...","[save, money, go, across, street, food, way, b..."
4,4.0,0,Oops...this was for the Main St location:\n\n-...,0,Q0QKTuceHzgmFeTtYSTzYg,0,2011-08-19 21:45:31,3zlK9eBVCpRWMPLjtAj6cQ,XtFhLvKV46H7a054X2_aFQ,oops...this was for the main st location:\n\n-...,oops...this was for the main st location:\n\n-...,oopsthis was for the main st location\n\n\n\nw...,"[oopsthis, was, for, the, main, st, location, ...","[oopsthis, main, st, location, visited, aladdi..."


# Step 6: Lemmatize/Stem
Stemming and Lemmatizing is the process of reducing a word to its root form. The main purpose is to reduce variations of the same word, thereby reducing the corpus of words we include in the model.

In [14]:
ps = PorterStemmer()

In [15]:
# a list to store stemmed words
stemmed_list = []

# loops through each sentence in the `text_non_punct_split_non_stopwords` column and then 
# creates a list 'stemmed_list' to store the stemmed version of each word 
for text in df['text_non_punct_split_non_stopwords']:
  stemmed = [ps.stem(word) for word in text]
  stemmed_list.append(stemmed)

df['title_non_punct_split_non_stopwords_stemmed'] = pd.Series(stemmed_list)
df.head()

,stars,useful,text,funny,review_id,cool,date,user_id,business_id,lower_case,text_non_digits,text_non_punct,text_non_punct_split,text_non_punct_split_non_stopwords,title_non_punct_split_non_stopwords_stemmed
0,4.0,0,First time waxing. I read all the other review...,0,V46usAbOSPIzurZy7DUU5A,0,2016-04-16 13:55:38,rD6noiuiMCcGuq6oPYFJdQ,b4NaB1u3AwgrhRMKsKgWsA,first time waxing. i read all the other review...,first time waxing. i read all the other review...,first time waxing i read all the other reviews...,"[first, time, waxing, i, read, all, the, other...","[first, time, waxing, read, reviews, decided, ...","[first, time, wax, read, review, decid, trust,..."
1,5.0,0,I love my manicure! Such an affordable place. ...,0,CpNSsju9NFc-Xrpf5Hfqhg,0,2020-01-18 05:32:54,tTsUSklzH1oxPp2usth84A,-otKp6RA94dfGsqa5jSDHQ,i love my manicure! such an affordable place. ...,i love my manicure! such an affordable place. ...,i love my manicure such an affordable place th...,"[i, love, my, manicure, such, an, affordable, ...","[love, manicure, affordable, place, manicurist...","[love, manicur, afford, place, manicurist, kin..."
2,4.0,0,"Great food, fast service, they try to crank pe...",0,RP7A24xD14Cbegmr1AokNQ,0,2015-01-25 14:49:09,-cnIq2OmV8CleS4bLOvCxQ,EA10YCkaxzRoCEOB38xv1Q,"great food, fast service, they try to crank pe...","great food, fast service, they try to crank pe...",great food fast service they try to crank peop...,"[great, food, fast, service, they, try, to, cr...","[great, food, fast, service, try, crank, peopl...","[great, food, fast, servic, tri, crank, peopl,..."
3,1.0,0,Save your money and go across the street the f...,0,6a38lS1TjeWMqK-knuNAhA,0,2018-11-06 22:12:14,mzCkRflEka_vezCiBYYWGA,nkqaLepNTSrUYuZkzbBR5A,save your money and go across the street the f...,save your money and go across the street the f...,save your money and go across the street the f...,"[save, your, money, and, go, across, the, stre...","[save, money, go, across, street, food, way, b...","[save, money, go, across, street, food, way, b..."
4,4.0,0,Oops...this was for the Main St location:\n\n-...,0,Q0QKTuceHzgmFeTtYSTzYg,0,2011-08-19 21:45:31,3zlK9eBVCpRWMPLjtAj6cQ,XtFhLvKV46H7a054X2_aFQ,oops...this was for the main st location:\n\n-...,oops...this was for the main st location:\n\n-...,oopsthis was for the main st location\n\n\n\nw...,"[oopsthis, was, for, the, main, st, location, ...","[oopsthis, main, st, location, visited, aladdi...","[oopsthi, main, st, locat, visit, aladdin, way..."


Next, we introduce the WordNetLemmatizer to perform lemmatization to the column 'text_non_punct_split_non_stopwords' to reduce the variations of words.

In [16]:
wnl = WordNetLemmatizer()

In [17]:
# a list to store lemmatization words
lemmatize_list = []

# loops through each sentence in the 'text_non_punct_split_non_stopwords' column and then
# stores words in the 'lemmatize_list' to store the lemmatized version of each word
for text in df['text_non_punct_split_non_stopwords']:
  lemmatize = [wnl.lemmatize(word) for word in text]
  lemmatize_list.append(lemmatize)
  
df['title_non_punct_split_non_stopwords_lemmatized'] = pd.Series(lemmatize_list)
df.head()

,stars,useful,text,funny,review_id,cool,date,user_id,business_id,lower_case,text_non_digits,text_non_punct,text_non_punct_split,text_non_punct_split_non_stopwords,title_non_punct_split_non_stopwords_stemmed,title_non_punct_split_non_stopwords_lemmatized
0,4.0,0,First time waxing. I read all the other review...,0,V46usAbOSPIzurZy7DUU5A,0,2016-04-16 13:55:38,rD6noiuiMCcGuq6oPYFJdQ,b4NaB1u3AwgrhRMKsKgWsA,first time waxing. i read all the other review...,first time waxing. i read all the other review...,first time waxing i read all the other reviews...,"[first, time, waxing, i, read, all, the, other...","[first, time, waxing, read, reviews, decided, ...","[first, time, wax, read, review, decid, trust,...","[first, time, waxing, read, review, decided, t..."
1,5.0,0,I love my manicure! Such an affordable place. ...,0,CpNSsju9NFc-Xrpf5Hfqhg,0,2020-01-18 05:32:54,tTsUSklzH1oxPp2usth84A,-otKp6RA94dfGsqa5jSDHQ,i love my manicure! such an affordable place. ...,i love my manicure! such an affordable place. ...,i love my manicure such an affordable place th...,"[i, love, my, manicure, such, an, affordable, ...","[love, manicure, affordable, place, manicurist...","[love, manicur, afford, place, manicurist, kin...","[love, manicure, affordable, place, manicurist..."
2,4.0,0,"Great food, fast service, they try to crank pe...",0,RP7A24xD14Cbegmr1AokNQ,0,2015-01-25 14:49:09,-cnIq2OmV8CleS4bLOvCxQ,EA10YCkaxzRoCEOB38xv1Q,"great food, fast service, they try to crank pe...","great food, fast service, they try to crank pe...",great food fast service they try to crank peop...,"[great, food, fast, service, they, try, to, cr...","[great, food, fast, service, try, crank, peopl...","[great, food, fast, servic, tri, crank, peopl,...","[great, food, fast, service, try, crank, peopl..."
3,1.0,0,Save your money and go across the street the f...,0,6a38lS1TjeWMqK-knuNAhA,0,2018-11-06 22:12:14,mzCkRflEka_vezCiBYYWGA,nkqaLepNTSrUYuZkzbBR5A,save your money and go across the street the f...,save your money and go across the street the f...,save your money and go across the street the f...,"[save, your, money, and, go, across, the, stre...","[save, money, go, across, street, food, way, b...","[save, money, go, across, street, food, way, b...","[save, money, go, across, street, food, way, b..."
4,4.0,0,Oops...this was for the Main St location:\n\n-...,0,Q0QKTuceHzgmFeTtYSTzYg,0,2011-08-19 21:45:31,3zlK9eBVCpRWMPLjtAj6cQ,XtFhLvKV46H7a054X2_aFQ,oops...this was for the main st location:\n\n-...,oops...this was for the main st location:\n\n-...,oopsthis was for the main st location\n\n\n\nw...,"[oopsthis, was, for, the, main, st, location, ...","[oopsthis, main, st, location, visited, aladdi...","[oopsthi, main, st, locat, visit, aladdin, way...","[oopsthis, main, st, location, visited, aladdi..."


# (2) List five words that got removed after removing stop words
1. 'i'
2. 'all'
3. 'the'
4. 'other'
5. 'and'

In [18]:
# Collects the stop words that are in the 'text_non_punct_split' but are removed from the 'text_non_punct_split_non_stopwords' column
words_removed = [words for words in df['text_non_punct_split'][0] if words not in df['text_non_punct_split_non_stopwords'][0]]
words_removed

['i',
 'all',
 'the',
 'other',
 'and',
 'to',
 'not',
 'it',
 'was',
 'and',
 'it',
 'was',
 'a',
 'i',
 'am',
 'it',
 'have',
 'been',
 'i',
 'will',
 'be']

# (3) Use two examples to briefly describe the purpose of 'lemmatzie'
1. Lemmatization aims to remove inflectional endings only and to return the base or dictionary form of a word. For example, in the following cells, the word 'reviews' is returned to 'review' after lemmatization.
2. Lemmatization also considers a language's full vocabulary to apply a morphological analysis to words. For example, 'are' is converted to 'be'.

In [19]:
# a list to store words that have been lemmatized
words_lemmatize = []

# loops through each sentences in the 'text_non_punct_split_non_stopwords' column and collects the words that not in 
# the 'title_non_punct_split_non_stopwords_lemmatized' column
for i in range(len(df)):
    words_lemma = [words for words in df['text_non_punct_split_non_stopwords'][i] if words not in df['title_non_punct_split_non_stopwords_lemmatized'][i]]
    words_lemmatize.append(','.join(words_lemma))

In [20]:
# Shows the changes in words after lemmatization
words_lemmatize[0:50]

['reviews',
 'hands,feet,lots,nails',
 '',
 'prices',
 'appetizers,lakes,portions,appetizers,pitas,juices,varieties,children,stars',
 'years,leaves,owners,dishes',
 'stars,tibs',
 'years',
 'grits,noodles,wraps,favorites',
 '',
 'lots,potatoes,prices,seams',
 'states,veggies,rolls,eyes',
 'years,fries',
 'gifts,accessories',
 '',
 'shops,drinks,beverages,items,lattes',
 '',
 'drinks,kinks,weeks',
 'years,days,amenities,beverages,robes,slippers,makes,services,aestheticians,changes',
 'manicures,pedicures,techs',
 'years,dogs,dogs,animals,needs,personalities,dogs,haircuts',
 'buns',
 'minutes',
 'rolls,adults',
 'rolls,ps',
 'cards,supports,businesses,lovers,things,cards,candles,masks,cards,stickers,spots,details,cards',
 'competitors,competitors',
 'renovations',
 'chips',
 'kids,kids,comes,kids',
 'women,fries,burgers',
 'migraines',
 'areas',
 'stars,tomatoes,tomatoes,portions',
 'kids,veggies,ingredients',
 'pitas,pastries',
 'ladies,chairs,ladies,eyebrows,eyebrows,eyebrows,ladies,ne

# Store pre-processed data in Google Drive and name it as 'yelp_review_cleaned.csv'

In [21]:
df.to_csv("/content/drive/MyDrive/INFOSYS765/yelp_review_cleaned.csv")